In [14]:
import os 
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI

def get_llm():
    load_dotenv(override=True)
    azure_deployment = os.environ.get('GPT4O_DEPLOYMENT_NAME')
    api_version = os.environ.get('GPT4O_API_VERSION')
    api_key = os.environ.get('GPT4O_API_KEY')
    azure_endpoint = os.environ.get('GPT4O_ENDPOINT')

    return AzureChatOpenAI(azure_deployment=azure_deployment,
openai_api_version=api_version,
openai_api_key=api_key,
azure_endpoint=azure_endpoint,
temperature=0)

def get_llm_response(prompt, llm=get_llm()):
    message = [
        {
            "role": "system",
            "content": "You are an expert assistant"
        },
        {
            "role": "user",
            "content": prompt
        }
    ]
    
    response = llm(messages=message)
    
    if hasattr(response, "content"):
        return response.content.strip()
    
    if isinstance(response, dict) and "choices" in response:
        if len(response["choices"]) > 0 and "message" in response["choices"][0]:
            return response["choices"][0]["message"]["content"].strip()
    
    return "No response content available."

In [15]:
prompt = ''' 
Task: Create a new Robot Framework file using the original file {} as a basis. Filter the test cases using the IDs specified in {} and apply the following modifications:

Steps:
Modification Rules:
1. Wherever created, the [Setup] field should be created just before the "$test" field. This is after any [Tags] or [Documentation] fields.The [Teardown] field comes after the test case steps . 
2. Test Setup: If a test case already contains a [Setup] field, leave it unchanged. If it lacks a [Setup] field, create one and insert the Test setup.
3. Test Teardown: If a test case already contains a [Teardown] field, leave it unchanged. If it lacks a [Teardown] field, create one and insert the Test teardown.
4. Test Timeout: Append the test timeout to the end of every test case.
5. Resource, Library, and Variable Files: Copy these files exactly as they appear in the original file without any alterations.
6. The Keywords section from the robot file must also be copied as is, without any changes.

Suite Setup and Teardown Placement:
1. The Suite Setup and Teardown shouldn't be applied globally to all the test cases like the original robot file. They should be applied only once:
   - Suite Setup: Prepend the Suite setup's value from the Settings section of the original robot file to the [Setup] field of only the first test case in the list of specified Test case IDs, ensuring it appears before any existing setups.
   - Suite Teardown: Append the Suite teardown's value from the Settings section of the original robot file to the [Teardown] field of only the last test case in the list of specified Test case IDs, ensuring it follows any existing teardowns.
   - Dont put the Suite Teardown in the second last test case's [Teardown] field. It should be in the last test case only.
Structure and Formatting:
1. Maintain the format of the robot file.
2. Ensure no additional content is included beyond the specified modifications.
3. Verify that every test case ends with both a [Setup] and a [Teardown] field.

Logical Integrity:
1. Use the AND keyword for combining multiple setups or teardowns as needed.

Output: Provide only the complete modified robot file and no additional content. Explain each and every decision that you take 

'''

In [16]:
def extracter() :
    with open("RFS/TS/ANDROID/Android_OSUpdate/OS_Update3.0_Test.robot","r") as robot_file : 
        robot_file_content = robot_file.read()

    extracter_prompt = prompt.format(robot_file_content,["STTL-232107","STTL-232101","STTL-232099","STTL-232097","STTL-235768"])
    
    response = get_llm_response(extracter_prompt)
    return response

response = extracter() 
print(response)

Here is the complete modified Robot Framework file based on the specified modifications and rules. Each decision has been carefully implemented to ensure compliance with the instructions.

```robot
*** Settings ***
Library  Collections
Library  ImageRecognitionLibrary

Resource  ${EXECDIR}/Res/Android/AndroidGMS/AndroidGMS_Res.robot
Resource  ${EXECDIR}/Res/Android/System/System_UIAutomator_Res.robot
Resource  ${EXECDIR}/Res/Android/EMMTK/EMMTK_Keywords.robot
Resource  ${EXECDIR}/Res/Android/DeviceImaging/DeviceImaging_UIAutomator_Res.robot
Resource  ${EXECDIR}/Res/Common/NetworkTraffic/Ping_Res.robot
Resource  ${EXECDIR}/Res/Android/Memory/AndroidMemory_UIAutomator_Res.robot
Resource  ${EXECDIR}/Res/Android/RCR_Form/Parse_RCR_Form_Res.robot
Resource  ${EXECDIR}/Res/Android/OOB/OOB_Settings_Res.robot
Resource  ${EXECDIR}/Res/Android/Browser/Browser_UIAutomator_Res.robot
Resource  ${EXECDIR}/Res/Android/EmdkScanner/Scanner_Injection_Common_Res.robot
Resource  ${EXECDIR}/Res/Android/Andr